In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale")

21/12/01 06:31:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-12-01 06:31:19, Running 1 Workers


In [2]:
df.count()

60000

In [3]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']
train_df, test_df = encoded.randomSplit([0.8, 0.2], seed=100)

In [4]:
from keras_tuner.engine import hyperparameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel

img_shape = (28, 28, 1)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="randomsearch", 
    hyperparameters=None, 
    objective="val_accuracy",
    max_trials=3,
    overwrite=True,
)

In [5]:
rel = am.fit(train_df, epochs=10, input_shape=img_shape)

Trial 1 Complete [00h 12m 52s]
val_accuracy: 0.9055159687995911

Best val_accuracy So Far: 0.9055159687995911
Total elapsed time: 00h 12m 52s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_block_1/ke...|3                 |3                 
conv_block_1/se...|False             |True              
conv_block_1/ma...|False             |True              
conv_block_1/dr...|0.5               |0                 
conv_block_1/nu...|1                 |1                 
conv_block_1/nu...|2                 |2                 
conv_block_1/fi...|256               |256               
conv_block_1/fi...|32                |32                
classification_...|global_avg        |flatten           
classification_...|0.25              |0.25              
optimizer         |adam_weight_decay |adam              
learning_rate     |0.0001            |2e-05             



Train on 1205 steps
 687/1205 [================>.............] - ETA: 50s - batch: 343.0000 - size: 1.0000 - loss: 2.3040 - accuracy: 0.0950 ETA: 1:54 - b - ETA: 1:41 - batc - ETA: 1:28 - batch: 143.5000 - size: 1.0000 - loss: 2.3046 - accuracy: 0.093 - ETA: 1:28 - batch: 144.5000 - size: 1.0000 - loss: 2.3046 - accuracy:  - ETA: 1:26 - batch: 154.0000 - size: 1.0000 - loss: 2.3043 - accuracy: - ETA: 1:24 - batch: 164.0000 - size: 1.0000 - loss: 2.3039 - accuracy - ETA: 1:22 - batch: 176.5000 - size: 1.0000 - loss: 2.3036 - accuracy: 0. - ETA: 1:21 - batch: 182.0000 - size: 1.0000 - loss: 2.3037 - accuracy: 0.09 - ETA: 1:20 - batch: 185.0000 - size: 1.0000  - ETA: 1:10 - batch: 239.5000 - size: 1.0000 - loss: 2.3040 - accuracy: 0.096 - ETA: 1:10 - batch: 240.5000 - size: 1.0000 - loss: 2.3040 - accu - ETA: 1:06 - batch: 258.00 - ETA: 55s - batch: 314.0000 - size: 1.0000 - loss: 2.3041 - accuracy - ETA: 54s - batch: 319.5000 - size: 1.0000 - los - ETA: 51s - batch: 337.5000 - size: 1.00

KeyboardInterrupt: 